In [1]:
import umap
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from scipy.stats import ranksums
from sklearn.preprocessing import StandardScaler

from core.constants import HEMATOSCOPE_BIOANALYST_COMPARISON_COLUMN_NAMES

from core.statistics.statistical_analysis import (
    chi_square_test,
    wilcoxon_rank_sum_test, 
    wilcoxon_rank_sum_test_with_collinearity_filter,
    summarize_continuous_variables_by_group,
    summarize_categorical_variables_by_group,
)
from core.preprocessing.data_cleansing import (
    clean_clinical_data,
    clean_wsi_data, 
    remove_absolute_variables,
    clean_sample_statistic_data,
)
from core.preprocessing.feature_engineering import (
    calculate_sample_cellularity,
    create_more_clinical_features,
    create_more_cell_count_features,
    get_tfr_modelling_data,
)
from core.utils import (
    categorize_based_on_median, 
    format_p_value, 
    transform_label, 
    disable_code_blocks,
)

from core.visualization.umap import points
from core.visualization.kaplan_meier import plot_kaplan_meier_curve
from core.visualization.boxplot import plot_covariate_distributions
from core.visualization.scatter_plot import draw_scatter_plot
from core.visualization.balloon_plot import plot_balloon_plot
from core.visualization.histogram import plot_histogram

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_colwidth', 100)

/home/HUSTIETOALLAS/ext11670944/.conda/envs/cml_tfr/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**A4 size in inches:** 8.3 x 11.7

In [2]:
disable_code_blocks

# Data loading and pre-processing

In [3]:
# MAIN COHORT
CLINICAL_DATA_PATH = None
WSI_IMAGE_DATA_PATH = None
CELL_COUNT_DATA_PATH = None
SAMPLE_STATISTICS_DATA_PATH = None
CELL_STATISTICS_DATA_PATH = None

# AUSTRALIA
AUS_HALVING_TIME_DATA_PATH = None
BIOANALYST_CELL_COUNT_DATA_PATH = None

# HEALTHY COHORT
HEALTHY_CELL_COUNT_DATA_PATH = None

# POOR RESPONSE COHORT
NON_RESPONDER_CELL_COUNT_DATA_PATH = None
NON_RESPONDER_SAMPLE_STATISTICS_DATA_PATH = None

In [5]:
# CLINICAL DATA 
clinical_data = pd.read_excel(CLINICAL_DATA_PATH, index_col='Hemavision ID')
clinical_data = create_more_clinical_features(clinical_data)
clinical_data = clean_clinical_data(clinical_data)

# WSI-IMAGE DATA
wsi_image_data = pd.read_excel(WSI_IMAGE_DATA_PATH, index_col='Hemavision ID')
valid_index_values = pd.Index(clinical_data.index).intersection(wsi_image_data.index)
wsi_image_data = wsi_image_data.loc[valid_index_values]
wsi_image_data = clean_wsi_data(wsi_image_data)
wsi_image_data = calculate_sample_cellularity(wsi_image_data)
                                
# CELL COUNT DATA
cell_count_data = pd.read_excel(CELL_COUNT_DATA_PATH, index_col='Hemavision ID')
cell_count_data = create_more_cell_count_features(cell_count_data)
cell_count_data = cell_count_data.drop(columns=cell_count_data.filter(regex='APL-Class|Vacuolated|Artefacts|Unknowns|Multinuclear|Erythroblasts-Class=').columns)

# SAMPLE STATISTIC DATA
statistics_data = pd.read_excel(SAMPLE_STATISTICS_DATA_PATH, index_col='Hemavision ID')
valid_index_values = pd.Index(clinical_data.index).intersection(statistics_data.index)
statistics_data = statistics_data.loc[valid_index_values]
statistics_data = clean_sample_statistic_data(statistics_data)

# CELL-LEVEL STATISTIC DATA
cell_statistics_data = pd.read_csv(CELL_STATISTICS_DATA_PATH)
# Remove one sample that had ~10 times more cells compared to other samples
cell_statistics_data = cell_statistics_data[~cell_statistics_data["Hemavision ID"].isin([5816])]
# remove redundant variables
cell_statistics_data = cell_statistics_data.drop(columns=cell_statistics_data.filter(regex='red|green|blue|mean|roundness|convexity').columns)
# create unique id/name for cells (image name + detection id) to identify cells from umap
cell_statistics_data['JPEG-DET-ID'] = cell_statistics_data["JPEG-ID"].astype(str) + "_" + cell_statistics_data["DET-IDX"].astype(str)
cell_statistics_data = cell_statistics_data.drop(columns=['DET-IDX', 'JPEG-ID'])
cell_statistics_data = cell_statistics_data[cell_statistics_data["Hemavision ID"].isin(clinical_data.index.to_list())]
cell_statistics_data = pd.merge(cell_statistics_data, clinical_data[['Relapse after first TKI discontinuation', 'Center', 'Major tyrosine kinase inhibitor is second-generation TKI']], on='Hemavision ID', how='left')

# BIOANALYST CELL COUNTS
bioanalyst_cell_count_data = pd.read_excel(BIOANALYST_CELL_COUNT_DATA_PATH, index_col='Hemavision ID')

# HALVING-TIME DATA
halving_time_data_aus = pd.read_excel(AUS_HALVING_TIME_DATA_PATH, index_col='Hemavision ID')[['3-month halving time']]

# HEALTHY SAMPLES
healthy_cell_count_data = pd.read_excel(HEALTHY_CELL_COUNT_DATA_PATH, index_col='Hemavision ID')
healthy_cell_count_data = healthy_cell_count_data.rename(columns={"Living_cells": "Living cells"})
healthy_cell_count_data = create_more_cell_count_features(healthy_cell_count_data)

# NON-RESPONDER DATA
non_responder_cell_count_data = pd.read_excel(NON_RESPONDER_CELL_COUNT_DATA_PATH, index_col='Hemavision ID')
non_responder_cell_count_data = create_more_cell_count_features(non_responder_cell_count_data)
non_responder_statistics_data = pd.read_excel(NON_RESPONDER_SAMPLE_STATISTICS_DATA_PATH, index_col='Hemavision ID')
# Remove columns with more than 50% NaN values from sample statistic data
non_responder_statistics_data = non_responder_statistics_data.loc[:, non_responder_statistics_data.isnull().mean() < 0.5]
non_responder_data = pd.concat([non_responder_cell_count_data, non_responder_statistics_data], axis=1, join='outer')

# merge all dataframes
multi_modal_data = pd.concat([clinical_data, wsi_image_data, cell_count_data, statistics_data], axis=1, join='inner')
multi_modal_data = pd.concat([multi_modal_data , halving_time_data_aus.reindex(multi_modal_data.index)], axis=1)
multi_modal_data = remove_absolute_variables(multi_modal_data)

# create dataframe where categorical variables has boolean data types
modelling_data, hemavision_ids = get_tfr_modelling_data(multi_modal_data)

# extract WSI variables
wsi_columns = wsi_image_data.columns.to_list()
WSI_VARIABLES = [col for col in wsi_columns if col in multi_modal_data.columns]

# extract cell count variables
cell_count_columns = cell_count_data.columns.to_list()
CELL_COUNT_VARIABLES = [col for col in cell_count_columns if col in multi_modal_data.columns]

# extract cell statistic variables
sample_statistic_columns = statistics_data.columns.to_list()
SAMPLE_STATISTIC_VARIABLES = [col for col in sample_statistic_columns if col in multi_modal_data.columns]

# all image variables
IMAGE_VARIABLES = WSI_VARIABLES + CELL_COUNT_VARIABLES + SAMPLE_STATISTIC_VARIABLES

PREDICTED_VARIABLE ='Relapse after first TKI discontinuation'
TIME_VARIABLE = 'Timepoint of possible relapse after first TKI discontinuation (months)'

# Analysis

### Relapse rates

**At 6 months after treatment discontinuation:**

In [ ]:
df = multi_modal_data[[PREDICTED_VARIABLE, TIME_VARIABLE]]
total_patients = len(df)
relapsed_within_6_months = df[(df[PREDICTED_VARIABLE] == 'yes') & (df[TIME_VARIABLE] <= 6)]
not_relapsed_within_6_months = total_patients - len(relapsed_within_6_months)
percent_relapsed_6_months = len(relapsed_within_6_months) / total_patients * 100
percent_not_relapsed_6_months = not_relapsed_within_6_months / total_patients * 100

print(f"  - Relapsed: {percent_relapsed_6_months:.1f}%")
print(f"  - Not Relapsed: {percent_not_relapsed_6_months:.1f}%")

**At the end:**

In [ ]:
df = multi_modal_data[[PREDICTED_VARIABLE, TIME_VARIABLE]]
relapsed_total = df[df[PREDICTED_VARIABLE] == 'yes']
not_relapsed_total = df[df[PREDICTED_VARIABLE] == 'no']
percent_relapsed_total = len(relapsed_total) / total_patients * 100
percent_not_relapsed_total = len(not_relapsed_total) / total_patients * 100
print(f"  - Relapsed: {percent_relapsed_total:.1f}%")
print(f"  - Not Relapsed: {percent_not_relapsed_total:.1f}%")

### Summary of TKIs

In [ ]:
def summarize_tki_types(df, TKI_VARIABLE):
    # Step 1: Get percentage distribution
    percentage_table = (
        df.groupby('Center')[TKI_VARIABLE]
        .value_counts(normalize=True)
        .mul(100)
        .rename('Percentage')
        .reset_index()
    )
    summary_table = percentage_table.pivot(index='Center', columns=TKI_VARIABLE, values='Percentage').fillna(0).round(2)
    display(summary_table)

#### First-line

In [ ]:
TKI_VARIABLE = "First-line tyrosine kinase inhibitor generic name before discontinuation"
df = multi_modal_data[["Center", TKI_VARIABLE]]
df = df[df[TKI_VARIABLE] != "Imatinib"]
summarize_tki_types(df, TKI_VARIABLE)

#### Last-line

In [ ]:
TKI_VARIABLE = "Last tyrosine kinase inhibitor generic name before discontinuation"
df = multi_modal_data[["Center", TKI_VARIABLE]]
df = df[df[TKI_VARIABLE] != "Imatinib"]
summarize_tki_types(df, TKI_VARIABLE)

### Patient characteristics

In [ ]:
numeric_variables = [
    "Patient age at diagnosis (completed years)",
    "BCR-ABL1 transcript level at 3 months (%)",
    "BCR-ABL1 transcript level at 6 months (%)",
    "BCR-ABL1 transcript level at 12 months (%)",
    "Time to MR4.0 (months)",
    "Duration of MR4.0 before first TKI discontinuation (months)",
    "Time to TKI discontinuation (months)",
]
summary_table_numeric = summarize_continuous_variables_by_group(
    multi_modal_data, 
    PREDICTED_VARIABLE, 
    numeric_variables, 
    {'no': 'Non-relapsing', 'yes': 'Relapse'},
    num_decimals=2
)
wilcoxon_results = wilcoxon_rank_sum_test(
    data=multi_modal_data, 
    predicted_variable=PREDICTED_VARIABLE, 
    variables=numeric_variables, 
    keep=True, 
    max_p_value=1,
)
patient_summary_numeric = summary_table_numeric.merge(wilcoxon_results[["covariate", "p-value"]], on='covariate')
patient_summary_numeric["p-value"] = patient_summary_numeric["p-value"].apply(format_p_value)

categorical_variables = [
    "Patient gender",
    "ELTS score-risk class", 
    "First-line tyrosine kinase inhibitor generic name before discontinuation",
    "First-line tyrosine kinase inhibitor is second-generation TKI",
    "Last tyrosine kinase inhibitor generic name before discontinuation",
    "Last tyrosine kinase inhibitor is second-generation TKI",
    "MR4.0 at 12 months",
    "Time to TKI discontinuation < 6 years",
]
df = multi_modal_data[[PREDICTED_VARIABLE] + categorical_variables]

summary_table_categorical = summarize_categorical_variables_by_group(
    multi_modal_data, 
    PREDICTED_VARIABLE, 
    categorical_variables,
    {'no': 'Non-relapsing', 'yes': 'Relapse'},
)
chi2_results = chi_square_test(
    data=multi_modal_data, 
    predicted_variable=PREDICTED_VARIABLE, 
    variables=categorical_variables,
    max_p_value=1,
)
patient_summary_categorical = summary_table_categorical.merge(chi2_results[["covariate", "p-value"]], on='covariate')
patient_summary_categorical["p-value"] = patient_summary_categorical["p-value"].apply(format_p_value)

patient_summary = pd.concat([patient_summary_categorical, patient_summary_numeric])
display(patient_summary)
#patient_summary.to_excel("./results/tables/cml-tfr_patient_summary.xlsx", index=False)

### Number of analyzed cells

#### Cell differential count data

In [ ]:
median = multi_modal_data["Living cells"].median()
q25 = multi_modal_data["Living cells"].quantile(0.25)
q75 = multi_modal_data["Living cells"].quantile(0.75)
print(f"Median of intact cells for differential counts: {int(median)} (range, {int(q25)} - {int(q75)})")

#### Cell statistic data

In [ ]:
df = cell_statistics_data[["CLS-LBL", "Hemavision ID"]].groupby("Hemavision ID").size()
median = df.median()
q25 = df.quantile(0.25)
q75 = df.quantile(0.75)
print(f"Median of intact cells for cell statistics: {int(median)} (range, {int(q25)} - {int(q75)})")

### UMAP of all the BM cells

In [ ]:
plot_umap = True

In [ ]:
hue = "Cell type"
hover_label = "JPEG-DET-ID"

if plot_umap:

    # Get cell-level statistics data
    df = cell_statistics_data.copy()
     
    # Exclude artefacts and unknows
    df = df[~df["CLS-LBL"].isin(["Artefacts", "Unknowns"])]
    # drop cells (rows) with missing feature data (missing columns)
    df = df.dropna()
    
    # Rename eosinophil maturity types
    df["CLS-LBL"] = df["CLS-LBL"].replace({
        'Eosinophils_Immature': 'Immature Eosinophils', 
        'Eosinophils_Mature': 'Mature Eosinophils'
    })
    
    # extract feature data
    X = df.drop(columns=[
        "CLS-LBL",
        "Hemavision ID",
        "Relapse after first TKI discontinuation", 
        "Center",
        "Major tyrosine kinase inhibitor is second-generation TKI",
        "JPEG-DET-ID",
    ])
    
    # extract label data
    label_data = {
        "Cell type": df["CLS-LBL"].to_numpy(),
        "JPEG-DET-ID": df["JPEG-DET-ID"].to_numpy(),
    }
    
    # Compute UMAP embeddings
    umap_reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.5, random_state=0)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_embedded = umap_reducer.fit_transform(X_scaled)

In [ ]:
if plot_umap:
    # Colors for visualization
    cell_colors = {
        # Erythropoiesis:
        'Proerythroblasts': 'maroon', 
        'Erythroblasts': 'lightcoral',
        'Megakaryocytes': 'darkorange', 
        # Granulopoiesis:
        'Blasts': 'tomato',
        'Promyelocytes': 'darkolivegreen',
        'Myelocytes': 'yellowgreen', 
        'Metamyelocytes': 'springgreen',
        'Basophils': 'tab:brown', 
        'Neutrophils': 'dodgerblue', 
        'Immature Eosinophils': 'mediumpurple',
        'Mature Eosinophils': 'blueviolet', 
        # Monocytopoiesis:
        'Promonocytes': 'navy',
        'Monocytes': 'gold',
        'Macrophages': 'darkkhaki',
        # Lymphopoiesis:
        'Lymphocytes': 'teal', 
        'Plasma cells': 'turquoise',
    }
    
    # Show UMAP-visualization
    fig, ax = plt.subplots(figsize=(4, 3))
    points(
        umap_reducer, 
        labels=label_data[hue], 
        color_key=cell_colors, 
        alpha=0.8, 
        fontsize=6, 
        save_path="./results/figures/umap_cells.png",
        ax=ax,
    )

<span style="font-size: 16px; font-weight: normal;"><strong>Extra: </strong>Previous UMAP with interactive hover labels for inspecting cell IDs.</span>

In [ ]:
import plotly.express as px

if plot_umap:
    
    fig = px.scatter(
        x=umap_reducer.embedding_[:, 0], 
        y=umap_reducer.embedding_[:, 1], 
        color=label_data[hue], 
        hover_name=label_data[hover_label], 
        width=700,
        height=500,
    )
    fig.update_traces(
        marker=dict(opacity=0.3),
        selector=dict(mode="markers"),
    )
    fig.show()

### BM cell differentials at diagnosis (healthy samples as a reference)

In [ ]:
variables_to_include = [
    # Erythropoiesis:
    'Proerythroblasts_Percentage',
    'Erythroblasts_Percentage',
    # Granulopoiesis:
    'Blasts_Percentage',
    'Promyelocytes_Percentage',
    'Myelocytes_Percentage', 
    'Metamyelocytes_Percentage',
    'Basophils_Percentage', 
    'Neutrophils_Percentage', 
    'Eosinophils_Percentage',
    # Monocytopoiesis:
    'Promonocytes_Percentage',
    'Monocytes_Percentage',
    'Macrophages_Percentage',
    # Lymphopoiesis:
    'Lymphocytes_Percentage', 
    'Plasma cells_Percentage',
]
df1 = multi_modal_data[variables_to_include]
df2 = healthy_cell_count_data[variables_to_include]

dfs = [df1, df2]
df_names = ['CP-CML', 'Healthy']

# prepare the data by melting each dataframe and adding a 'DataFrame_ID' to distinguish them
melted_dfs = []
for df, name in zip(dfs, df_names):
    df_copy = df.copy().reset_index(drop=True)  # Ensure no index conflict
    df_copy['DataFrame_ID'] = name  # Add the DataFrame identifier to avoid naming conflict
    melted = pd.melt(df_copy, id_vars='DataFrame_ID', var_name='Cell type', value_name='Proportion (%)')
    melted_dfs.append(melted)

# Combine all melted dataframes into one
combined_df = pd.concat(melted_dfs, ignore_index=True)

# Create a figure and axis
fig, ax = plt.subplots(1, 1, figsize=(4, 2.8))
fontsize = 7

# Plot the data for each dataframe
sns.stripplot(
    y='Cell type', x='Proportion (%)', data=combined_df,
    hue='DataFrame_ID', dodge=True, palette={'CP-CML': 'tab:red', 'Healthy': 'tab:blue'},
    size=2, linewidth=0, zorder=0, alpha=0.7, ax=ax
)
sns.boxplot(
    y='Cell type', x='Proportion (%)', data=combined_df,
    hue='DataFrame_ID', dodge=True, palette={'CP-CML': 'tab:red', 'Healthy': 'tab:blue'},
    linewidth=0.8, ax=ax, showfliers=False, whis=0, zorder=1,
    boxprops=dict(facecolor='none', edgecolor='black'),
    whiskerprops=dict(color='black', linestyle='--'),
    capprops=dict(color='black', linestyle='--'),
    medianprops=dict(color='black')
)

# generate significance-marked y-axis labels
significant_labels = []
for cell in variables_to_include:
    values_df = df1[cell].dropna()
    values_df2 = df2[cell].dropna()
    if len(values_df) > 1 and len(values_df2) > 1 and values_df.nunique() > 1 and values_df2.nunique() > 1:
        stat, p = ranksums(values_df, values_df2)
        if p <= 0.001:
            stars = "***"
        elif p <= 0.01:
            stars = "**"
        elif p <= 0.05:
            stars = "*"
        else:
            stars = ""
        significant_labels.append(f"{cell.split('_')[0]} {stars}")

ax.set_yticklabels(significant_labels, fontsize=fontsize-1)
ax.set_ylabel("")
ax.set_xlabel("Percentage (%)", fontsize=fontsize-1)
ax.set_xticks([0, 5, 10, 20, 30, 40, 50, 60])
ax.tick_params(axis='x', labelsize=fontsize-1)
ax.set_xlim(-1, 70)

ax.spines[['left', 'right', 'top']].set_visible(False)
ax.tick_params(left=False)

# Add legend
handles, labels = ax.get_legend_handles_labels()
handles = [mpatches.Patch(color='tab:red', label='CP CML'), mpatches.Patch(color='tab:blue', label='Healthy')]
ax.legend(handles=handles, loc='best', frameon=False, fontsize=fontsize-1)

ax.set_title("Bone marrow differential cell counts \nin CP CML and healthy reference cohorts", fontsize=fontsize)

plt.tight_layout()
plt.savefig("./results/figures/box_strip_plot_cell_differential_counts.png", format='png', dpi=300)
plt.show()

#### Summary of CP CML differential counts (Median [25%-75%]):

In [ ]:
variables_to_include = [
    # Erythropoiesis:
    'Proerythroblasts_Percentage',
    'Erythroblasts_Percentage',
    'Erythropoietic cells_Percentage',
    # Granulopoiesis:
    'Blasts_Percentage',
    'Promyelocytes_Percentage',
    'Myelocytes_Percentage', 
    'Metamyelocytes_Percentage',
    'Basophils_Percentage', 
    'Neutrophils_Percentage', 
    'Eosinophils_Percentage',
    'Immature granulopoietic cells_Percentage',
    'Mature granulopoietic cells_Percentage',
    'Granulopoietic cells_Percentage',
    # Monocytopoiesis:
    'Promonocytes_Percentage',
    'Monocytes_Percentage',
    'Macrophages_Percentage',
    # Lymphopoiesis:
    'Lymphocytes_Percentage', 
    'Plasma cells_Percentage',
]
'Immature granulopoietic cells_Percentage'
'Mature granulopoietic cells_Percentage'
'Granulopoietic cells_Percentage'

df = multi_modal_data[variables_to_include]
summary = pd.DataFrame({
    'Median': df.median(),
    '25%': df.quantile(0.25),
    '75%': df.quantile(0.75),
}).round(1)
display(summary)

#### Summary of healthy control differential counts (Median [25%-75%]):

In [ ]:
df = healthy_cell_count_data[variables_to_include]
summary = pd.DataFrame({
    'Median': df.median(),
    '25%': df.quantile(0.25),
    '75%': df.quantile(0.75),
}).round(1)
display(summary)

### Association of cytomorphological variables with TFR

#### Statistical association measured with Wilcoxon rank sum test

**Note:** Inspect the multicollinearity of the most significant variables to detect variables that might describe the same phenomenon. We use spearman correlation coefficient for the comparison. If variable is part of cell statistics, we will compare the variable to other 

In [ ]:
wilcoxon_results = wilcoxon_rank_sum_test_with_collinearity_filter(
    df=multi_modal_data,
    predicted_variable=PREDICTED_VARIABLE, 
    sample_statistic_variables=SAMPLE_STATISTIC_VARIABLES, 
    image_variables=IMAGE_VARIABLES,
    variables=IMAGE_VARIABLES,
    keep=True,
    correlation_threshold=0.8,
    max_p_value=0.05
)
# drop cell shape related variables because visual inspection revealed those to be technical signal
wilcoxon_results = wilcoxon_results[~wilcoxon_results['covariate'].str.contains('_cell-', na=False)]

summary = summarize_continuous_variables_by_group(
    multi_modal_data, 
    PREDICTED_VARIABLE, 
    wilcoxon_results['covariate'].to_list(), 
    labels = {'yes': 'Relapsed', 'no': 'Remission'}, 
    num_decimals=3,
)

wilcoxon_results = wilcoxon_results.merge(summary, on="covariate")
wilcoxon_results

#### Box plots of the statistically significant cytomorphological variables

In [ ]:
covariates_to_plot = {
    "Promyelocytes_nuclei-eccentricity_median": "Promyelocyte nuclei \n ellipticity (Mdn)",
    "Eosinophils_Immature_Percentage": "Immature \n eosinophils (%)",
    "Eosinophils_Immature_to_Mature granulopoietic cells_Ratio": "Ratio of immature \n eosinophils to \n granulocytes",
    "Neutrophils_Percentage": "Neutrophils (%)",
    "Neutrophils_nuclei-solidity_std": "Neutrophil nuclei \n solidity (σ)",
    "ME-ratio": "M:E ratio",
}
plot_covariate_distributions(
    multi_modal_data, 
    covariates_to_plot, 
    PREDICTED_VARIABLE, 
    label_mapping={"no": "Non-\nrelapsing", "yes": "Relapse"},
    custom_palette={"no": "tab:blue", "yes": "tab:red"}, 
    fontsize=7, 
    num_columns=6, 
    fig_width=8.3,
    fig_height=2.5,
    save_path="./results/figures/box_strip_plot_cytomorphological variables associated with tfr.png",
)

#### Correlation between M:E Ratio and granulopoietic cells (%)

In [ ]:
draw_scatter_plot(
    x_variable="ME-ratio",
    y_variable="Granulopoietic cells_Percentage",
    df=multi_modal_data,
    correlation_method='spearman',
    show_trendline=False,
    show_regression=False,
    show_correlation=True,
    x_label="M:E ratio",
    y_label="Granulopoietic cells (%)",
    show_grid=False,
    figsize=(2.5, 2.5),
    fontsize=7,
)

### Kaplan-meier analysis of the cytomorphological variables

#### Promyelocyte nuclei eccentricity (Mdn)

In [ ]:
VARIABLE = "Promyelocytes_nuclei-eccentricity_median"

df = modelling_data.copy()
df = df[[TIME_VARIABLE, VARIABLE, PREDICTED_VARIABLE]].dropna()
df, median = categorize_based_on_median(df, VARIABLE, below_median=False, decimals=2)

plot_kaplan_meier_curve(
    df=df,
    time_col=TIME_VARIABLE,
    event_col=PREDICTED_VARIABLE,
    group_col="Promyelocytes_nuclei-eccentricity_median > median",
    fontsize=7,
    custom_legends = {"False": f"Ellipticity $\leq${median}", "True": f"Ellipticity $>${median}"},
    colors = {"False": "tab:blue", "True": "tab:red"},
    title="Probability of sustained TFR based on \n the ellipticity of promyelocyte nuclei",
    xlabel="Months after TKI discontinuation",
    ylabel="Probability of TFR",
    figsize = (2.5, 1.8),
    save_path="./results/figures/kaplan_meier_promyelocyte_nuclei_ellipticity.png",
)

#### M:E Ratio

In [ ]:
VARIABLE = "ME-ratio"

df = modelling_data.copy()
df = df[[TIME_VARIABLE, VARIABLE, PREDICTED_VARIABLE]].dropna()
df, median = categorize_based_on_median(df, VARIABLE, below_median=False, decimals=1)

plot_kaplan_meier_curve(
    df=df,
    time_col=TIME_VARIABLE,
    event_col=PREDICTED_VARIABLE,
    group_col="ME-ratio > median",
    fontsize=7,
    custom_legends = {"True": f"M:E $>${median}", "False": f"M:E $\leq${median}"},
    colors = {"True": "tab:blue", "False": "tab:red"},
    title="Probability of sustained TFR based on \nthe M:E ratio",
    xlabel="Months after TKI discontinuation",
    ylabel="Probability of TFR",
    figsize = (2.5, 1.8),
    save_path="./results/figures/kaplan_meier_ME-ratio.png",
)

#### Neutrophils (%)

In [ ]:
VARIABLE = "Neutrophils_Percentage"

df = modelling_data.copy()
df = df[[TIME_VARIABLE, VARIABLE, PREDICTED_VARIABLE]].dropna()
df, median = categorize_based_on_median(df, VARIABLE, below_median=False, decimals=1)

plot_kaplan_meier_curve(
    df=df,
    time_col=TIME_VARIABLE,
    event_col=PREDICTED_VARIABLE,
    group_col="Neutrophils_Percentage > median",
    fontsize=7,
    custom_legends = {"True": f"$>${median} %", "False": f"$\leq${median} %"},
    colors = {"True": "tab:blue", "False": "tab:red"},
    title="Probability of sustained TFR based on \n the neutrophil percentage",
    xlabel="Months after TKI discontinuation",
    ylabel="Probability of TFR",
    figsize = (2.5, 1.8),
    save_path="./results/figures/kaplan_meier_neutrophils.png",
)

#### Ratio of immature eosinophils to granulocytes

In [ ]:
VARIABLE = "Eosinophils_Immature_to_Mature granulopoietic cells_Ratio"

df = modelling_data.copy()
df = df[[TIME_VARIABLE, VARIABLE, PREDICTED_VARIABLE]].dropna()
df, median = categorize_based_on_median(df, VARIABLE, below_median=False, decimals=2)

plot_kaplan_meier_curve(
    df=df,
    time_col=TIME_VARIABLE,
    event_col=PREDICTED_VARIABLE,
    group_col="Eosinophils_Immature_to_Mature granulopoietic cells_Ratio > median",
    fontsize=7,
    custom_legends = {"False": f"Ratio $\leq${median}", "True": f"Ratio $>${median}"},
    colors = {"False": "tab:blue", "True": "tab:red"},
    title="Probability of sustained TFR based on the ratio \nof immature eosinophils to granulocytes",
    xlabel="Months after TKI discontinuation",
    ylabel="Probability of TFR",
    save_path="./results/figures/kaplan_meier_imm-eos-to-gran.png",
    figsize = (2.5, 1.8),
)

### Association of cytomorphology and prognostic variables in CML

In [ ]:
prognostic_variables = {
    "Spleen size at diagnosis (max. distance from costal margin) > 0": "Spleen size $>$0 cm",
    "Time to TKI discontinuation < 6 years": "Time to TKI discontinuation $<$6 years",
    "Duration of MR4.0 before first TKI discontinuation (months) < median": "Duration of MR4.0 $<$42 months",
    "MR4.0 at 12 months": "MR4.0 at 12 months",
    "Last tyrosine kinase inhibitor is second-generation TKI": "Last-line 2G-TKI",
    "First-line tyrosine kinase inhibitor is second-generation TKI": "First-line 2G-TKI ",
    "ELTS score-risk class_low": "ELTS, low",
    "Patient age at diagnosis > 50": "Age $>$50 years",
    "Patient gender_male": "Gender, male",
}
covariates_to_plot = {
    "Neutrophils_Percentage": "Neutrophils (%)",
    "ME-ratio": "M:E Ratio",
    "Neutrophils_nuclei-solidity_std": "Neutrophil nuclei solidity (σ)",
    "Promyelocytes_nuclei-eccentricity_median": "Promyelocyte nuclei ellipticity (Mdn)", 
    "Eosinophils_Immature_Percentage": "Immature eosinophils (%)",
    "Eosinophils_Immature_to_Mature granulopoietic cells_Ratio": "Ratio of immature eosinophils to granulocytes", 
}

data = modelling_data.copy()
data, median_value = categorize_based_on_median(data, "Duration of MR4.0 before first TKI discontinuation (months)", below_median=True)

plot_balloon_plot(
    df=data,
    cat_vars=prognostic_variables,
    cont_vars=covariates_to_plot,
    max_pvalue=0.05,
    plot_height=2.5,
    plot_width=2.5,
    fontsize=7,
    title="Association of cytomorphological variabes \n with prognostic variables in CML", 
    save_path="./results/figures/balloon_plot_prognostic_variables_vs_cytomorphology.png",
)

### Analysis of ELTS

In [ ]:
covariates_to_plot = {
    "ME-ratio": "M:E ratio",
    #"Granulopoietic cells_Percentage": "Granulopoietic cells (%)",
    #"Erythropoietic cells_Percentage": "Erythropoietic cells (%)",
}

for variable, title in covariates_to_plot.items():
    plot_covariate_distributions(
        modelling_data.dropna(subset="ELTS score-risk class_low"),
        {variable: title}, 
        "ELTS score-risk class_low", 
        label_mapping={True: "low", False: "intermediate-\nhigh"},
        custom_palette={False: "tab:red", True: "tab:blue"}, 
        xlabel="ELTS",
        fontsize=7, 
        num_columns=1, 
        fig_width=1.5, 
        fig_height=2.5,
        save_path=f"./results/figures/box_strip_plot_{variable}-vs-ELTS.png",
    )

### Analysis of spleen size

In [ ]:
covariates_to_plot = {
    "ME-ratio": "M:E ratio",
    "Eosinophils_Immature_to_Mature granulopoietic cells_Ratio": "Ratio of immature \n eosinophils to \n granulopoietic cells",
}

for variable, label in covariates_to_plot.items():
    plot_covariate_distributions(
        multi_modal_data.dropna(subset="Spleen size at diagnosis (max. distance from costal margin) > 0"), 
        {variable: label}, 
        "Spleen size at diagnosis (max. distance from costal margin) > 0", 
        label_mapping={"no": "Normal", "yes": "Enlarged"},
        custom_palette={"no": "tab:blue", "yes": "tab:red"}, 
        xlabel="Spleen size",
        fontsize=7, 
        num_columns=1, 
        fig_width=1.5, 
        fig_height=2.5,
        save_path=f"./results/figures/box_strip_plot_{variable}-vs-spleen size.png",
    )

In [ ]:
draw_scatter_plot(
    x_variable="Granulopoietic cells_Percentage",
    y_variable="Spleen size at diagnosis (max. distance from costal margin)",
    df=multi_modal_data,
    correlation_method='pearson',
    show_trendline=False,
    show_regression=False,
    show_correlation=False,
    title="Relationship between spleen size \n and granulopoietic cells",
    x_label="Granulopoietic cells (%)",
    y_label="Spleen size (cm)",
    figsize=(2.2, 2.5),
    fontsize=7,
    marker_size=10,
    save_path=f"./results/figures/scatter_plot_granulopoietic cells-vs-spleen size.png",
)

### Analysis of halving time

#### Association of halving time and TFR

In [ ]:
data = multi_modal_data.copy()
data = data.dropna(subset=["3-month halving time"])

plot_covariate_distributions(
    data, 
    {"3-month halving time": "Halving time (days)"}, 
    PREDICTED_VARIABLE, 
    label_mapping={"no": "Non-relapsing", "yes": "Relapse"},
    custom_palette={"no": "tab:blue", "yes": "tab:red"}, 
    fontsize=7, 
    num_columns=1, 
    fig_width=1.5, 
    fig_height=2.5,
    save_path=f"./results/figures/box_strip_plot_3-month halving time.png",
)

#### Kaplan-Meier analysis of halving time

In [ ]:
VARIABLE = "3-month halving time"

df = modelling_data.copy()
df = df[[TIME_VARIABLE, VARIABLE, PREDICTED_VARIABLE]].dropna()
df, median = categorize_based_on_median(df, VARIABLE, below_median=False, decimals=2)

plot_kaplan_meier_curve(
    df=df,
    time_col=TIME_VARIABLE,
    event_col=PREDICTED_VARIABLE,
    group_col="3-month halving time > median",
    fontsize=7,
    custom_legends = {"False": f"$\leq${median} days", "True": f"$>${median} days"},
    colors = {"False": "tab:blue", "True": "tab:red"},
    title="Probability of sustained TFR based on \n the halving time",
    xlabel="Months after TKI discontinuation",
    ylabel="Probability of TFR",
    figsize = (2.5, 1.8),
    save_path=f"./results/figures/kaplan_meier_3-month halving time.png",
)

#### Association of cytomorphology and halving time

In [ ]:
covariates_to_plot = {
    "Promyelocytes_nuclei-eccentricity_median": "Promyelocyte \n nuclei ellipticity (Mdn)",
    "Eosinophils_Immature_Percentage": "Immature \n eosinophils (%)",
    "Eosinophils_Immature_to_Mature granulopoietic cells_Ratio": "Ratio of \n immature eosinophils \n to granulocytes",
    "Neutrophils_Percentage": "Neutrophils (%)",
    "Neutrophils_nuclei-solidity_std": "Neutrophil \n nuclei solidity (σ)",
    "ME-ratio": "M:E ratio",
}

data = modelling_data.copy()
data = data.dropna(subset=["3-month halving time"])
halving_time_median = round(data["3-month halving time"].median(), 1)
data["3-month halving time > median"] = data["3-month halving time"] > halving_time_median

plot_covariate_distributions(
    data.dropna(subset="3-month halving time > median"), 
    covariates_to_plot, 
    "3-month halving time > median", 
    label_mapping={False: f"$\leq${halving_time_median}", True: f"$>${halving_time_median}"},
    custom_palette={False: "tab:blue", True: "tab:red"}, 
    xlabel="Halving time",
    fontsize=7, 
    num_columns=7, 
    fig_width=10, 
    fig_height=2.5,
)

### Analysis of poor response samples

In [ ]:
covariates_to_plot = {
    "Blasts_Percentage": "Blasts (%)",
    "ME-ratio": "M:E ratio",
    "Erythropoietic cells_Percentage": "Erythropoietic cells (%)", 
    "Neutrophils_Percentage": "Neutrophils (%)",
    "Eosinophils_Immature_to_Mature granulopoietic cells_Ratio":"Ratio of immature eosinophils \n to granulocytes",
    "Eosinophils_Immature_Percentage":"Immature eosinophils (%)",
    "Promyelocytes_nuclei-eccentricity_median": "Promyelocyte \n nuclei ellipticity (Mdn)",
    "Neutrophils_nuclei-solidity_std": "Neutrophil \n nuclei solidity (σ)",
}

df_cp = multi_modal_data[list(covariates_to_plot.keys()) + [PREDICTED_VARIABLE, TIME_VARIABLE]]
df_cp["Category"] = df_cp[PREDICTED_VARIABLE].replace({"yes": "Relapse", "no": "Remission"})
df_cp = df_cp.drop(columns=PREDICTED_VARIABLE)

df_cp.loc[df_cp[TIME_VARIABLE] <= 6, "Category"] = "Early relapse"
df_cp.loc[df_cp[TIME_VARIABLE] > 6, "Category"] = "Late relapse"

df_pr = non_responder_data[list(covariates_to_plot.keys())]
df_pr.index = df_pr.index.map(lambda x: int(f"{x}00"))
df_pr["Category"] = "Non-responder"

df_merged = pd.concat([df_cp, df_pr], ignore_index=False)

plot_covariate_distributions(
    df_merged,
    covariates_to_plot, 
    "Category",
    label_mapping={"Remission": "Non-\nrelapsing", "Late relapse": "Late\n relapse", "Early relapse": "Early\n relapse", "Non-responder": "Non-\n    responder"},
    custom_palette={"Remission": "tab:blue", "Late relapse": "tab:orange", "Early relapse": "tab:red", "Non-responder": "w"}, 
    fontsize=7, 
    num_columns=4,
    fig_width=8.3,
    fig_height=5,
    show_trend_line=True,
    save_path="./results/figures/box_strip_plot_4-group comparison.png",
)

### BM differentical count comparison (hematoscope vs. bioanalyst)

**Number of analyzed cells with Hematoscope:**

In [ ]:
multi_modal_data_aus = pd.concat([clinical_data, bioanalyst_cell_count_data, cell_count_data], axis=1, join='inner')
multi_modal_data_aus = multi_modal_data_aus.rename(columns=HEMATOSCOPE_BIOANALYST_COMPARISON_COLUMN_NAMES)

median = multi_modal_data_aus["Living cells"].median()
q25 = multi_modal_data_aus["Living cells"].quantile(0.25)
q75 = multi_modal_data_aus["Living cells"].quantile(0.75)
print(f"Median of living cells: {int(median)} (range, {int(q25)} - {int(q75)})")

#### Erythropoietic cells (%)

In [ ]:
draw_scatter_plot(
    x_variable="Erythropoietic cells (%) - Hematoscope",
    y_variable="Erythropoietic cells (%) - Human",
    df=multi_modal_data_aus,
    correlation_method='pearson',
    show_trendline=True,
    show_regression=False,
    show_correlation=True,
    figsize=(2.5, 2.5),
    fontsize=7,
    title="Comparison of Hematoscope and morphologist \ndetermined erythropoietic cell percentage",
    x_label="Erythropoietic cells (%), Hematoscope",
    y_label="Erythropoietic cells (%), Morphologist",
    save_path="./results/figures/scatter_plot_erythropoietic-cells_hematoscope-vs-human.png",
)

#### M:E Ratio

In [ ]:
draw_scatter_plot(
    x_variable="M:E Ratio - Hematoscope",
    y_variable="M:E Ratio - Human",
    df=multi_modal_data_aus,
    correlation_method='pearson',
    show_trendline=True,
    show_regression=False,
    show_correlation=True,
    figsize=(2.5, 2.5),
    fontsize=7,
    title="Comparison of Hematoscope and \nmorphologist determined M:E ratio",
    x_label="M:E ratio, Hematoscope",
    y_label="M:E ratio, Morphologist",
    save_path="./results/figures/scatter_plot_ME-ratio_hematoscope-vs-human.png",
)

### Comparison of promyelocytes nuclei eccentricity values of non-relapsing and relapsed patients

**Pick** example sample from Australia with remission status and low median promyelocyte nuclei eccentricity:

In [ ]:
VARIABLE = "Promyelocytes_nuclei-eccentricity_median"
df = multi_modal_data.copy()
df = df[(df[PREDICTED_VARIABLE] == "no") & (df["Center"] == "Australia")][[VARIABLE]].sort_values(by=VARIABLE, ascending=True)
display(df.head(3))

**Pick** example sample from Australia with relapse status and high median promyelocyte nuclei eccentricity:

In [ ]:
VARIABLE = "Promyelocytes_nuclei-eccentricity_median"
df = multi_modal_data.copy()
df = df[(df[PREDICTED_VARIABLE] == "yes") & (df["Center"] == "Australia")][[VARIABLE]].sort_values(by=VARIABLE, ascending=False)
display(df.head(3))

In [ ]:
df = cell_statistics_data.copy()
df = df[df["CLS-LBL"] == "Promyelocytes"]
df = df[df["Hemavision ID"].isin([349, 326])]
df[PREDICTED_VARIABLE] = df[PREDICTED_VARIABLE].map({"yes": "Relapsed", "no": "Non-relapsing"})

plot_histogram(
    df=df,
    dependent_variable=PREDICTED_VARIABLE,
    x_variable="nuclei-eccentricity",  # or any other column name
    palette={"Relapsed": "tab:red", "Non-relapsing": "tab:blue"},
    bins=10,
    kde=True,
    line_width=3,
    xlabel="Nuclei ellipticity",
    ylabel="Cell count",
    title="Comparison of nuclei ellipticity in promyelocytes\n between a relapsed and a non-relapsing patient",
    figsize=(3.8, 2.5),
    save_path="./results/figures/histogram_promyelocyte_nuclei_eccentricity.png",
)

### Comparison of neutrophil nuclei solidity values of non-relapsing and relapsed patients

**Pick** example sample from Australia with remission status and low neutrophil nuclei solidity variance (standard deviation):

In [ ]:
VARIABLE = "Neutrophils_nuclei-solidity_std"
df = multi_modal_data.copy()
df = df[(df[PREDICTED_VARIABLE] == "yes") & (df["Center"] == "Australia")][[VARIABLE]].sort_values(by=VARIABLE, ascending=True)
display(df.head(1))

**Pick** example sample from Australia with relapse status and high neutrophil nuclei solidity variance (standard deviation):

In [ ]:
VARIABLE = "Neutrophils_nuclei-solidity_std"
df = multi_modal_data.copy()
df = df[(df[PREDICTED_VARIABLE] == "no") & (df["Center"] == "Australia")][[VARIABLE]].sort_values(by=VARIABLE, ascending=False)
display(df.head(1))

In [ ]:
df = cell_statistics_data.copy()
df = df[df["CLS-LBL"] == "Neutrophils"]
df = df[df["Hemavision ID"].isin([321, 338])]
df[PREDICTED_VARIABLE] = df[PREDICTED_VARIABLE].map({"yes": "Relapsed", "no": "Non-relapsing"})

plot_histogram(
    df=df,
    dependent_variable=PREDICTED_VARIABLE,
    x_variable="nuclei-solidity",
    palette={"Relapsed": "tab:red", "Non-relapsing": "tab:blue"},
    bins=10,
    kde=True,
    line_width=2,
    xlabel="Nuclei solidity",
    ylabel="Cell count",
    title="Comparison of nuclei solidity in neutrophils\n between a relapsed and a non-relapsing patient",
    figsize=(3.8, 2.5),
    save_path="./results/figures/histogram_neutrophils_nuclei_solidity.png",
)

### Neutrophil nuclei solidity: Standard deviation vs. median

In [ ]:
draw_scatter_plot(
    x_variable="Neutrophils_nuclei-solidity_median",
    y_variable="Neutrophils_nuclei-solidity_std",
    df=multi_modal_data,
    correlation_method='spearman',
    show_trendline=False,
    show_regression=True,
    show_correlation=True,
    title="Correlation between the median \nand standard deviation \nof neutrophil nuclei solidity",
    x_label="Nuclei solidity (Mdn)",
    y_label="Nuclei solidity (σ)",
    show_grid=False,
    figsize=(2.5, 2.5),
    fontsize=7,
    save_path="./results/figures/scatter_plot_neutrophil_nuclei_solidity.png",
)

In [ ]:
VARIABLE = "Neutrophils_nuclei-solidity_std"
df = multi_modal_data.copy()
df, _ = categorize_based_on_median(df, VARIABLE, below_median=False)

plot_covariate_distributions(
    df, 
    {"Neutrophils_nuclei-solidity_median": "Neutrophil \n nuclei solidity (Mdn)"}, 
    f"{VARIABLE} > median", 
    label_mapping={True: f"> Median", False: f"$\leq$ Median"},
    custom_palette={True: "tab:blue", False: "tab:red"}, 
    fontsize=7,
    xlabel=f"Neutrophil nuclei solidity (σ)",
    num_columns=1, 
    fig_width=1.75, 
    fig_height=2.5,
    save_path="./results/figures/box_strip_plot_neutrophil_nuclei_solidity.png",
)

### Association of cell statistic and cell differential counts

#### Erythroblasts

In [ ]:
CELL_TYPE = "Erythroblasts"
dependent_variable = f"{CELL_TYPE}_Percentage > median"
data = multi_modal_data[IMAGE_VARIABLES]
data, _ = categorize_based_on_median(data, f"{CELL_TYPE}_Percentage", below_median=False)

wilcoxon_results = wilcoxon_rank_sum_test(
    data=data, 
    predicted_variable=dependent_variable, 
    variables=SAMPLE_STATISTIC_VARIABLES,
    keep=True, 
    max_p_value=0.05,
)
wilcoxon_results = wilcoxon_results[
    (wilcoxon_results["covariate"].str.startswith(CELL_TYPE)) &
    (wilcoxon_results["p-value"] <= 0.05)
]

covariates = wilcoxon_results["covariate"].to_list()
tested_variables = {variable: transform_label(variable, keep_cell_type=False)  for variable in covariates}

for variable, title in tested_variables.items():
    plot_covariate_distributions(
        data, 
        {variable: title}, 
        f"{CELL_TYPE}_Percentage > median", 
        label_mapping={False: f"$\leq$Median", True: f"$>$Median"},
        custom_palette={False: "tab:blue", True: "tab:red"}, 
        fontsize=7,
        xlabel=f"{CELL_TYPE} (%)",
        num_columns=1, 
        fig_width=1.5, 
        fig_height=2,
        save_path=f"./results/figures/box_strip_plot_{variable}.png",
    )

In [ ]:
draw_scatter_plot(
    x_variable="Erythroblasts_cell-solidity_median",
    y_variable="Erythroblasts_cell-solidity_std",
    df=multi_modal_data,
    correlation_method='spearman',
    show_trendline=False,
    show_regression=True,
    show_correlation=True,
    x_label="Cell solidity (Mdn)",
    y_label="Cell solidity (σ)",
    show_grid=False,
    figsize=(2, 2),
    fontsize=7,
    title="Correlation between the median \nand standard deviation \n of erythroblast solidity",
    save_path=f"./results/figures/scatter_plot_erythroblast_cell_solidity.png",
)

In [ ]:
draw_scatter_plot(
    x_variable="Erythroblasts_cytoplasm-area_median",
    y_variable="Erythroblasts_cytoplasm-area_std",
    df=multi_modal_data,
    correlation_method='spearman',
    show_trendline=False,
    show_regression=True,
    show_correlation=True,
    title="Correlation between the median \nand standard deviation of \nerythroblast cytoplasm area",
    x_label="Cytoplasm area (Mdn)",
    y_label="Cytoplasm area (σ)",
    show_grid=False,
    figsize=(2, 2),
    fontsize=7,
    save_path=f"./results/figures/scatter_plot_Erythroblasts_cytoplasm-area.png",
)

#### Promyelocytes

In [ ]:
VARIABLE = "Promyelocytes_nuclei-eccentricity_median"
df = multi_modal_data.copy()
df = df.dropna(subset=VARIABLE)
df, _ = categorize_based_on_median(df, VARIABLE, below_median=False)

summarize_continuous_variables_by_group(
    df,
    f"{VARIABLE} > median", 
    ["Promyelocytes_Percentage"], 
    labels = {True: f"> Median", False: f"$\leq$ Median"}, 
    num_decimals=3,
)

#### Immature eosinophils

In [ ]:
CELL_TYPE = "Eosinophils_Immature"

dependent_variable = {f"{CELL_TYPE}_Percentage > median": f"Immature Eosinophils (%) \n > Median"}
data = multi_modal_data[IMAGE_VARIABLES]
data, _ = categorize_based_on_median(data, f"{CELL_TYPE}_Percentage", below_median=False)

wilcoxon_results = wilcoxon_rank_sum_test(
    data=data, 
    predicted_variable=list(dependent_variable.keys())[0], 
    variables=SAMPLE_STATISTIC_VARIABLES,
    keep=True, 
    max_p_value=0.05,
)
wilcoxon_results = wilcoxon_results[
    (wilcoxon_results["covariate"].str.startswith(CELL_TYPE)) &
    (wilcoxon_results["p-value"] <= 0.05)
]
covariates = wilcoxon_results["covariate"].to_list()
tested_variables = {variable: transform_label(variable, keep_cell_type=False)  for variable in covariates}

plot_covariate_distributions(
    data, 
    tested_variables,
    f"{CELL_TYPE}_Percentage > median", 
    label_mapping={False: f"$\leq$ Median", True: f"> Median"},
    custom_palette={False: "tab:blue", True: "tab:red"}, 
    fontsize=7,
    xlabel="Immature Eosinophils (%)",
    num_columns=5, 
    fig_width=7, 
    fig_height=2,
    save_path=f"./results/figures/box_strip_plot_eosinophils_immature.png",
)

In [ ]:
draw_scatter_plot(
    x_variable="Eosinophils_Immature_nuclei-area_median",
    y_variable="Eosinophils_Immature_cell-area_median",
    df=multi_modal_data,
    correlation_method='pearson',
    show_trendline=False,
    show_regression=True,
    show_correlation=True,
    x_label="Nuclei area (Mdn)",
    y_label="Cell area (Mdn)",
    figsize=(2.5, 2.5),
    fontsize=7,
    title="Correlation between the median \narea of nuclei and the whole cell \nof immature eosinophils",
    save_path=f"./results/figures/scatter_plot_Eosinophils_Immature_area.png",
)

In [ ]:
draw_scatter_plot(
    x_variable="Eosinophils_Immature_nuclei-perimeter_median",
    y_variable="Eosinophils_Immature_cell-perimeter_median",
    df=multi_modal_data,
    correlation_method='pearson',
    show_trendline=False,
    show_regression=True,
    show_correlation=True,
    x_label="Nuclei perimeter (Mdn)",
    y_label="Cell perimeter (Mdn)",
    figsize=(2.5, 2.5),
    fontsize=7,
    title="Correlation between the median \nperimeter of nuclei and the whole cell \nof immature eosinophils",
    save_path=f"./results/figures/scatter_plot_Eosinophils_Immature_perimeter.png",
)

#### Neutrophils

No significant variables found.